In [7]:
pip install catboost lightgbm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.8/27.8 MB 8.6 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 7.3 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [123]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor, Pool 
from lightgbm import LGBMRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import make_scorer, mean_squared_error, mean_absolute_error, root_mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV

In [125]:
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

In [148]:
# Загрузка данных
activities = pd.read_csv('chembl262_bioactivity_cleaned.csv')
rdkit = pd.read_csv('rdkit_data.csv')
maccs = pd.read_csv('maccs_data.csv')

rdkit['pIC50'] = activities['pIC50']
maccs['pIC50'] = activities['pIC50']

data = pd.merge(rdkit, maccs, on='smiles', how='inner')
data = data.drop(['smiles'], axis=1)
rdkit = rdkit.drop(['smiles'], axis=1)
maccs = maccs.drop(['smiles'], axis=1)
data.rename(columns={'pIC50_y': 'pIC50'}, inplace=True)
data = data.drop(['pIC50_x'], axis=1)

In [150]:
data.head()

,MaxAbsEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MaxPartialCharge,MinPartialCharge,FpDensityMorgan1,AvgIpc,BalabanJ,...,maccs_157,maccs_158,maccs_159,maccs_160,maccs_162,maccs_163,maccs_164,maccs_165,maccs_166,pIC50
0,13.007274,0.016808,-4.424338,0.546081,14.083333,0.415934,-0.354218,1.083333,2.790726,2.029523,...,0,1,0,0,1,1,1,1,0,5.70000
1,13.119549,0.107024,-0.630714,0.526419,13.555556,0.419020,-0.443178,1.185185,2.785592,2.089523,...,1,1,1,1,1,1,1,1,0,5.64000
2,12.800172,0.152321,-0.333605,0.509350,14.514286,0.324350,-0.492262,1.028571,3.254698,1.367027,...,1,1,1,1,1,1,1,1,0,4.30103
3,11.769662,0.035658,0.035658,0.764233,14.600000,0.228201,-0.462728,1.350000,3.033978,1.688217,...,0,1,1,0,1,1,1,1,0,6.85000
4,12.531840,0.266685,-0.460966,0.691295,14.520000,0.259139,-0.349902,0.920000,3.044437,2.057259,...,0,1,1,1,1,1,1,1,0,7.63500


In [152]:
def evaluate_model(y_true, y_pred):
    """Вычисление метрик качества модели."""
    return {
        'MAE': mean_absolute_error(y_true, y_pred),
        'RMSE': root_mean_squared_error(y_true, y_pred),
        'R2': r2_score(y_true, y_pred)
    }

$R^2$, или коэффициент детерминации (Coefficient of Determination), — это статистическая мера, которая используется для оценки того, насколько хорошо регрессионная модель (такая как LightGBM, CatBoost или нейронная сеть в нашем случае) объясняет вариацию зависимой переменной (в нашем случае pIC50).

Простыми словами:
* **$R^2$ показывает, какую долю изменчивости (дисперсии) в ваших фактических данных может объяснить модель.**
* Он измеряет, насколько хорошо предсказания модели соответствуют реальным значениям.


**Как это работает?**

Представьте, что у вас есть набор истинных значений pIC50 для разных соединений. Эти значения варьируются (имеют какую-то дисперсию). Ваша цель - построить модель, которая бы объяснила эту вариацию.
* Низкий $R^2$ (близкий к 0) означает, что ваша модель объясняет очень малую часть вариации в данных. По сути, предсказания вашей модели не сильно лучше, чем если бы вы просто предсказывали среднее значение всех ваших pIC50.
* Высокий $R^2$ (близкий к 1) означает, что ваша модель объясняет большую часть вариации в данных. Предсказания модели очень близки к истинным значениям, и модель хорошо улавливает основные тенденции в данных.

Сперва обучим модели Random Forest отдельно на каждом датасете, потом обучим на объединенном из двух, сравним метрики

In [154]:
X_rdkit = rdkit.drop(['pIC50'], axis=1)
y_rdkit = rdkit['pIC50']

In [156]:
# Разделение данных на обучающую и тестовую выборки 80/20
X_train_rdkit, X_test_rdkit, y_train_rdkit, y_test_rdkit = train_test_split(
    X_rdkit, y_rdkit, test_size=0.2, random_state=SEED
)

In [158]:
# Random Forest
rf_rdkit = RandomForestRegressor(n_estimators=200, max_depth=15, random_state=SEED, n_jobs=-1)
rf_rdkit.fit(X_train_rdkit, y_train_rdkit)
rf_rdkit_pred = rf_rdkit.predict(X_test_rdkit)
rf_rdkit_metrics = evaluate_model(y_test_rdkit, rf_rdkit_pred)
print(rf_rdkit_metrics)

{'MAE': 0.6048601805516628, 'RMSE': 0.8191252026730592, 'R2': 0.6291165262012757}


In [160]:
X_maccs = maccs.drop(['pIC50'], axis=1)
y_maccs = maccs['pIC50']

In [162]:
# Разделение данных на обучающую и тестовую выборки 80/20
X_train_maccs, X_test_maccs, y_train_maccs, y_test_maccs = train_test_split(
    X_maccs, y_maccs, test_size=0.2, random_state=SEED
)

In [164]:
# Random Forest
rf_maccs = RandomForestRegressor(n_estimators=200, max_depth=15, random_state=SEED, n_jobs=-1)
rf_maccs.fit(X_train_maccs, y_train_maccs)
rf_maccs_pred = rf_maccs.predict(X_test_maccs)
rf_maccs_metrics = evaluate_model(y_test_maccs, rf_maccs_pred)
print(rf_maccs_metrics)

{'MAE': 0.5857285999047258, 'RMSE': 0.8149562625517691, 'R2': 0.6328821442469912}


In [166]:
X = data.drop(['pIC50'], axis=1)
y = data['pIC50']

In [168]:
# Разделение данных на обучающую и тестовую выборки 80/20
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=SEED
)

In [170]:
# Random Forest
rf = RandomForestRegressor(random_state=SEED, n_jobs=-1)
rf.fit(X_train, y_train)
rf_pred = rf.predict(X_test)
rf_metrics = evaluate_model(y_test, rf_pred)
print(rf_metrics)

{'MAE': 0.5865325994597586, 'RMSE': 0.8011260639682504, 'R2': 0.6452367474644367}


In [172]:
# XGBoost
xgb = XGBRegressor(random_state=SEED, n_jobs=-1, eval_metric='rmse')
xgb.fit(X_train, y_train)
xgb_pred = xgb.predict(X_test)
xgb_metrics = evaluate_model(y_test, rf_pred)
print(xgb_metrics)

{'MAE': 0.5865325994597586, 'RMSE': 0.8011260639682504, 'R2': 0.6452367474644367}


In [174]:
# LightGBM
lgmr = LGBMRegressor(random_state=42, n_jobs=-1)
lgmr.fit(X_train, y_train)
lgmr_pred = lgmr.predict(X_test)
lgmr_metrics = evaluate_model(y_test, lgmr_pred)
print(lgmr_metrics)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005864 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11184
[LightGBM] [Info] Number of data points in the train set: 3073, number of used features: 209
[LightGBM] [Info] Start training from score 6.193294
{'MAE': 0.5790984020502693, 'RMSE': 0.7868309524924877, 'R2': 0.6577844205214576}


In [175]:
# CatBoost
cbr = CatBoostRegressor(random_state=SEED, verbose=0), # verbose=0, чтобы не печатать логи CatBoost
cbr.fit(X_train, y_train, eval_set=(X_test, y_test), early_stopping_rounds=50, verbose=0)
cbr_pred = cbr.predict(X_test)
cbr_metrics = evaluate_model(y_test, cbr_pred)
print(cbr_metrics)

AttributeError: 'tuple' object has no attribute 'fit'

In [177]:
# LinearRegression
lr = LinearRegression()
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)
lr_metrics = evaluate_model(y_test, lr_pred)
print(lr_metrics)

{'MAE': 0.7976693933219681, 'RMSE': 1.0383987651446884, 'R2': 0.40397396415104025}


Проведем поиск по сетку для подрбора гиперпараметров LightGBM, CatBoost

In [180]:
# Определяем диапазоны гиперпараметров для LightGBM
param_distributions_lgbm = {
    'n_estimators': [100, 200, 300, 500, 700, 1000],          # Количество деревьев
    'learning_rate': [0.005, 0.01, 0.05, 0.1, 0.15, 0.2],     # Скорость обучения
    'num_leaves': [10, 20, 31, 40, 50, 70],                   # Максимальное количество листьев в дереве
    'max_depth': [-1, 5, 10, 15, 20],                         # Максимальная глубина дерева (-1 означает отсутствие ограничения)               
    'reg_alpha': [0, 0.1, 0.5, 1, 2],                         # L1 регуляризация
    'reg_lambda': [0, 0.1, 0.5, 1, 2]                         # L2 регуляризация
}

# Определяем диапазоны гиперпараметров для CatBoost
param_distributions_catboost = {
    'iterations': [100, 200, 300, 500, 700, 1000],            # Количество итераций (деревьев)
    'learning_rate': [0.005, 0.01, 0.05, 0.1, 0.15, 0.2],     # Скорость обучения
    'depth': [4, 6, 8, 10],                                   # Глубина дерева
    'l2_leaf_reg': [1, 2, 3, 5, 7],                           # L2 регуляризация
    'border_count': [32, 64, 128, 254]                        # Количество порогов для дискретизации числовых признаков
}

In [182]:
# Инициализация моделей
model_lgbm = LGBMRegressor(random_state=SEED)
catboost_model = CatBoostRegressor(random_state=42, verbose=0, allow_writing_files=False)

In [184]:
# Настройка RandomSearchCV СatBoost
random_search_catboost = RandomizedSearchCV(
    estimator=catboost_model,
    param_distributions=param_distributions_catboost,
    n_iter=30, # Количество случайных комбинаций
    cv=3,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    random_state=SEED,
    verbose=1
)

In [186]:
# Настройка RandomSearchCV LightGBM
random_search_lgbm = RandomizedSearchCV(
    estimator=model_lgbm,
    param_distributions=param_distributions_lgbm,
    n_iter=30, # Количество случайных комбинаций для тестирования
    scoring='neg_mean_squared_error', 
    cv=3,  # Кросс-валидация
    verbose=2,
    random_state=SEED,
    n_jobs=-1  # Использовать все ядра процессора
)

In [188]:
# Запуск поиска
random_search_catboost.fit(X_train, y_train)

Fitting 3 folds for each of 30 candidates, totalling 90 fits


RandomizedSearchCV(cv=3,
                   estimator=<catboost.core.CatBoostRegressor object at 0x16459ae10>,
                   n_iter=30, n_jobs=-1,
                   param_distributions={'border_count': [32, 64, 128, 254],
                                        'depth': [4, 6, 8, 10],
                                        'iterations': [100, 200, 300, 500, 700,
                                                       1000],
                                        'l2_leaf_reg': [1, 2, 3, 5, 7],
                                        'learning_rate': [0.005, 0.01, 0.05,
                                                          0.1, 0.15, 0.2]},
                   random_state=42, scoring='neg_mean_squared_error',
                   verbose=1)

In [189]:
print(f"Лучшие параметры для CatBoost: {random_search_catboost.best_params_}")
print(f"Лучший RMSE для CatBoost: {np.sqrt(-random_search_catboost.best_score_):.4f}")

Лучшие параметры для CatBoost: {'learning_rate': 0.1, 'l2_leaf_reg': 5, 'iterations': 700, 'depth': 8, 'border_count': 64}
Лучший RMSE для CatBoost: 0.8218


In [190]:
best_catboost_model = random_search_catboost.best_estimator_

y_pred_catboost_final = best_catboost_model.predict(X_test)
mse_catboost_final = mean_squared_error(y_test, y_pred_catboost_final)
catboost_final_metrics = evaluate_model(y_test, y_pred_catboost_final)
print(catboost_final_metrics)

{'MAE': 0.5707573171211368, 'RMSE': 0.7765858479504532, 'R2': 0.6666381872238953}


In [192]:
# Запуск поиска
random_search_lgbm.fit(X_train, y_train)

Fitting 3 folds for each of 30 candidates, totalling 90 fits
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.027896 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 10847
[LightGBM] [Info] Number of data points in the train set: 2049, number of used features: 202
[LightGBM] [Info] Start training from score 6.200409
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightG

RandomizedSearchCV(cv=3, estimator=LGBMRegressor(random_state=42), n_iter=30,
                   n_jobs=-1,
                   param_distributions={'learning_rate': [0.005, 0.01, 0.05,
                                                          0.1, 0.15, 0.2],
                                        'max_depth': [-1, 5, 10, 15, 20],
                                        'n_estimators': [100, 200, 300, 500,
                                                         700, 1000],
                                        'num_leaves': [10, 20, 31, 40, 50, 70],
                                        'reg_alpha': [0, 0.1, 0.5, 1, 2],
                                        'reg_lambda': [0, 0.1, 0.5, 1, 2]},
                   random_state=42, scoring='neg_mean_squared_error',
                   verbose=2)

In [193]:
print(f"Лучшие параметры для LightGBM: {random_search_lgbm.best_params_}")
print(f"Лучший RMSE для LightGBM: {np.sqrt(-random_search_lgbm.best_score_):.4f}")

Лучшие параметры для LightGBM: {'reg_lambda': 2, 'reg_alpha': 0.5, 'num_leaves': 50, 'n_estimators': 200, 'max_depth': 15, 'learning_rate': 0.05}
Лучший RMSE для LightGBM: 0.8148


In [194]:
best_lgbm_model = random_search_lgbm.best_estimator_ # Это уже обученная модель с лучшими параметрами
y_pred_lgbm_final = best_lgbm_model.predict(X_test)
lgbm_final_metrics = evaluate_model(y_test, y_pred_lgbm_final)
print(lgbm_final_metrics)

{'MAE': 0.5732588653160585, 'RMSE': 0.7890591713011591, 'R2': 0.6558434422448093}


In [195]:
# Инициализация и обучение ФИНАЛЬНОЙ LightGBM модели на ВСЕМ ДАТАСЕТЕ
final_lgbm_model = LGBMRegressor(**random_search_lgbm.best_params_, random_state=SEED, n_jobs=-1)
final_lgbm_model.fit(X, y) # Обучаем на всем data
print("Финальная LightGBM модель успешно обучена на всем датасете.")

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.005997 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 11320
[LightGBM] [Info] Number of data points in the train set: 3842, number of used features: 210
[LightGBM] [Info] Start training from score 6.199266
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Финальная LightGBM модель успешно обучена на всем датасете.


In [196]:
# Инициализация и обучение ФИНАЛЬНОЙ CatBoost модели на ВСЕМ ТРЕНИРОВОЧНОМ ДАТАСЕТЕ ---
final_catboost_model = CatBoostRegressor(**random_search_catboost.best_params_, random_state=42, verbose=0, allow_writing_files=False)
final_catboost_model.fit(X, y) # Обучаем на всем data
print("Финальная CatBoost модель успешно обучена на всем датасете.")

Финальная CatBoost модель успешно обучена на всем датасете.


In [197]:
# Сохранение ФИНАЛЬНЫХ моделей, обученных на всем датасете
joblib.dump(final_lgbm_model, 'final_lgbm_model.pkl')
joblib.dump(final_catboost_model, 'final_catboost_model.pkl')
joblib.dump(X.columns.tolist(), 'descriptor_columns.pkl')
print("\nФинальные модели LightGBM и CatBoost (обученные на всем датасете) сохранены.")


Финальные модели LightGBM и CatBoost (обученные на всем датасете) сохранены.
